In [4]:
def recalc_feldspar(path,sheet_name):
    """
    READ ME
    
    This will recalculate olivine analysis and propagate errors with a standard error analysis for An and Ab
    
    Inputs:
        path = some path name to your file. This must be input in parentheses and end in .xlsx
        Your file headers must be in all lower case. For instance, SiO2 must be sio2, etc.
        Your first column must be the name that you gave your spot when measuring
        sheet_name = input the sheet name of that excel file where your data lives. This must also be in parentheses. 
        
    """
    #import dependables
    import pandas as pd
    import numpy as np
    import math
    from statistics import stdev
    df = pd.read_excel(path,sheet_name)
    #set up a general dataframe that has a number of oxides, their number of O2-, the cation:anion, and the molecular weights
    oxides = pd.DataFrame([[60.0855,101.963,94.1966,56.087,79.867,70.938,71.845,61.9796,40.305,141.9476,151.990,74.693,64.066,153.326,149.881,265.81,81.379,18.998,35.453,80.063],
                          [2,3,1,1,2,1,1,1,1,5,3,1,2,1,3,5,1,1,1,3],
                          [1/2,2/3,2,1,1/2,1,1,2,1,2/5,2/3,1,1/2,1,2/3,2/5,1,1,1,1/3]],
                      columns=['sio2','al2o3','k2o','cao','tio2','mno','feo','na2o','mgo','p2o5','cr2o3','nio','so2','bao','v2o3','nb2o5','zno','f','cl','so3'])
    #pull out the column with strings
    spots = df.iloc[:,0]
    #pull out all the columns with data in them
    data = df.iloc[:,1:]
    #set any negative numbers = 0
    data[data < 0] = 0
    #put together the measured data and the general oxides dataframe, dropping any columns from 'oxides' that does not exist in the measured dataset 
    newframe = pd.concat([data,oxides], join='inner')
    
    #the line below does a few things:
    #1. gives you the molar proportion of oxides by dividing measurements by molecular weights
    #2. reorganizes the dataframes to retain the oxides dataframe in a neat manner
    #3. adds a suffix for the eventual output dataframe accordingly
    newframe = pd.concat([newframe.iloc[:-3,:].div(newframe.iloc[-3,:]),newframe.iloc[-3:]]).reset_index().drop('index',axis=1).add_suffix('_molar proportion')

    #multiply by the number of oxygens per oxide in order to get the atomic proportion of oxygen
    atomico2 = newframe.iloc[:-3,:].multiply(newframe.iloc[-2,:])
    #add up the atomic proportion of oxygen. This helps you normalize your measurement in a step below
    atomico2['number o2-'] = atomico2.sum(axis=1)
    #make a normalizing factor for each analysis based on an ideal number of oxygen with respect to what you actually measured
    atomico2['normalizing factor'] = 32 / atomico2['number o2-']
    #put the oxides information back in for ease of calculating and cleanliness
    atomico2 = pd.concat([atomico2,newframe.iloc[-3:]])

    #get the amount of negative charge that is required for each of the cations you measured. This is effectively how your ideal number of negative charges (which you've obtained by normalizing) are distributed about your mineral site based on the cations that are actually measureable.
    anions = atomico2.iloc[:-3,:-2].multiply(atomico2.iloc[:-3,-1],axis=0)
    #sum the total number of negative charges. This should sum up to the numerator you have used above. For the case of olivine, this would be 4.
    anions['negative charge total'] = anions.sum(axis=1)
    #put the oxides information back in for ease of calculating and cleanliness
    anions = pd.concat([anions,newframe.iloc[-3:]])
    #change the suffix at the end of this series of columns for outputting later
    anions.columns = anions.columns.str.replace('_molar proportion','_charge distribution')
    
    #multiply the distributed oxygen charge by the ratio of cation to anion. A simple unit analysis should guide your intution
    cations = anions.iloc[:-3,:-1].multiply(anions.iloc[-1,:-1])
    #sum the number of cations. This should be pretty close to your mineral stoichiometry. For olivine, that is 3 (i.e., (Fe,Mg)2SiO4)
    cations['sum cations'] = cations.sum(axis=1)
    #change the suffix for hte purposes of outputting later
    cations.columns = cations.columns.str.replace('_charge distribution','_as cation')
    #calculate the sum of those cations that contribute to the endmember of the mineral
    cations['M sites Total'] = cations['cao_as cation']+cations['na2o_as cation']+cations['k2o_as cation']
    #Fa content
    cations['An'] = cations['cao_as cation'] / cations['M sites Total']
    #Fo content
    cations['Ab'] = cations['na2o_as cation'] / cations['M sites Total']
    #Tp content
    cations['Or'] = cations['k2o_as cation'] / cations['M sites Total']

    #use error propagation to get the error on your Fo. If you are the type to monitor your standards, you could use this to get your analytical error. Note this does NOT include error of individual analyses and that is for you to decide whether to use or not; this is simply an error analysis based on your standard deviation. 
    cations['An error propagated'] = cations['An'].mean() * ((stdev(cations['cao_as cation'])/(cations['cao_as cation'].mean()))**2 + (((stdev(cations['cao_as cation']))**2+(stdev(cations['na2o_as cation'])**2)+(stdev(cations['k2o_as cation'])**2))**(1/2)/(cations['M sites Total'].mean()))**2)**(1/2)
    #as above, but for Fa
    cations['Or error propagated'] = cations['Or'].mean() * ((stdev(cations['k2o_as cation'])/(cations['k2o_as cation'].mean()))**2 + (((stdev(cations['cao_as cation']))**2+(stdev(cations['na2o_as cation'])**2)+(stdev(cations['k2o_as cation'])**2))**(1/2)/(cations['M sites Total'].mean()))**2)**(1/2)

    #concatenate all the above information and fill non-existing values with a 0
    plagioclase_recalc = pd.concat([spots,data,atomico2,anions,cations],axis=1).fillna(0)
    #output the dataframe to an excel file
    output = plagioclase_recalc.to_excel('plagioclase_recalculated.xlsx')
    
    return output

In [5]:
recalc_feldspar('/Users/ctlewis/Documents/Projects/Caspana_052921/Phases/Feldspar_plinandrhy_07122020.xlsm','py_code')